In [40]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
import pyspark.sql.functions as func
from pyspark.sql.types import *

%run insights.py

In [41]:
spark = SparkSession.builder.getOrCreate()

In [100]:
metrics = spark.read.load('../data/dataset_metrics.parquet').toPandas()
comments = spark.read.load('../data/sample.parquet')
comments.registerTempTable("comments")

### Dataset Description

##### Source

The Dataset contains all the comments available on Reddit parsed by...

##### Overview

Let's start with the basic information about the dataset, i.e. its size and range of dates.

In [84]:
insights_comments(metrics)

> The dataset contains a total of **3,092,028,928** comments ranging from **12/12/2005** to **03/31/2017**.

Before we go further, it is also important to understand what values we have. Here is the list of all available fields for each comment.

In [98]:
comments.printSchema()

root
 |-- distinguished: string (nullable = true)
 |-- retrieved_on: string (nullable = true)
 |-- gilded: integer (nullable = true)
 |-- edited: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- flair_text: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- ups: integer (nullable = true)
 |-- downs: integer (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_context: string (nullable = true)
 |-- flair_css_class: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- archived: boolea

There are two important things here. First, we do not have access to the content of the posts, only the comments. We do however have access to the id of the post (`link_id`) on which the comments are written. We can use this information, for instance to group comments that related to the same post. It is also possible to calculate the total number of posts (with at least one reply) on Reddit.

In [85]:
insights_posts(metrics)

> There is a total of **204,469,276** posts on Reddit with at least one reply.

Moreover, it is also important to understand the notion of *thread* in Reddit comments. Each comment has a parent, defined by `parent_id`. It means that comments might be at different depth. Top-level comments (which have depth 0 and answer the original post directly) will have the post id as `parent_id`. Lower-level comments on the other hand, have another comment as parent. We can use this information to extract the number of top-level comments and lower-level comments from the dataset.

In [92]:
insights_comments_levels(metrics)

> Out of all the comments, **1,223,260,504** (**39.56%** of the dataset) are top-level comments. This makes an average of **5.98** top-level comments per post. The other **1,868,768,424** (**60.44%**) are replies to other comments, with a maximum depth of **[MISSING: max_depth]**.

##### Deleted and Removed Comments

Some comments are marked as `[deleted]` or `[removed]`. The former means that the comment was simply deleted by its author. There are a few reasons to do that, for instance to hide a negative comment from one's history, or to avoid getting more downvotes that we already have and prevent one's karma from getting too low. 

In [78]:
insights_removed(metrics)

> A total of **186,592,666** comments were deleted by their authors, which is **6.03%** of the dataset.

On the other hand, `[deleted]` indicates comments that were removed from the moderation team of the subreddit that the comment belongs to. This can also indicate potential toxic or vulgar comments, but it is not very reliable as it can also include off-topic comments.

In [38]:
insights_deleted(metrics)

> A total of **25,544,034** comments were removed by the moderation teams, which is **0.83%** of the dataset.

##### Score

On Reddit, people can *upvote* (`ups` in the dataset) posts or comments that they agree with, and *downvote* (`downs`) comments with which they do not agree. These values are then used to calculate a `score` metric with a simple formula `ups - downs`. 

Unfortunately for us, even though the dataset sometimes include `ups` and `downs` attributes, their value seems to be completely wrong. In fact, there is no instance where `downs` is bigger than 0.

In [103]:
spark.sql("""SELECT downs, COUNT(*) FROM comments GROUP BY downs""").show()

+-----+--------+
|downs|count(1)|
+-----+--------+
| null| 2637603|
|    0| 3543945|
+-----+--------+



This is problematic for us, as we were hoping to use the number of upvotes and downvotes of a comment as a means to measure its controversiality.

In [ ]:
df = metrics.toPandas()

In [ ]:
df['count_of_deleted'][0]

In [111]:
metrics = spark.read.load('../data/daily_metrics_2.parquet')

In [112]:
metrics.toPandas().tail(20)

,created,count_of_comments,count_of_users,count_of_posts,count_of_subreddits,count_of_top_comments,count_of_child_comments,prop_of_top_comments,prop_of_child_comments,total_score,...,prop_of_top_comments_60d_avg,prop_of_child_comments_60d_avg,total_score_60d_avg,avg_score_60d_avg,total_gold_60d_avg,prop_of_gold_60d_avg,count_of_removed_60d_avg,prop_of_removed_60d_avg,count_of_controversial_60d_avg,prop_of_controversial_60d_avg
3486,2017-03-12,248,232,242,192,101,147,0.407258,0.592742,1322,...,0.396737,0.603263,2024.360000,7.721693,0.040000,0.000184,6.680000,0.026025,4.640000,0.017772
3487,2017-03-13,260,247,251,186,110,150,0.423077,0.576923,1359,...,0.395920,0.604080,2036.387755,7.767073,0.040816,0.000187,6.755102,0.026322,4.673469,0.017900
3488,2017-03-14,274,248,267,201,102,172,0.372263,0.627737,1956,...,0.396622,0.603378,2037.750000,7.767223,0.041667,0.000191,6.770833,0.026378,4.708333,0.018027
3489,2017-03-15,277,259,271,204,115,162,0.415162,0.584838,1408,...,0.394470,0.605530,2041.702128,7.757353,0.042553,0.000195,6.787234,0.026372,4.702128,0.017937
3490,2017-03-16,284,264,280,220,114,170,0.401408,0.598592,1847,...,0.394817,0.605183,2018.304348,7.655941,0.043478,0.000200,6.804348,0.026425,4.630435,0.017635
3491,2017-03-17,303,279,299,218,115,188,0.379538,0.620462,3073,...,0.395038,0.604962,2028.000000,7.699615,0.044444,0.000204,6.755556,0.026293,4.600000,0.017547
3492,2017-03-18,196,182,193,151,75,121,0.382653,0.617347,768,...,0.395515,0.604485,2028.636364,7.701115,0.045455,0.000209,6.681818,0.026023,4.568182,0.017425
3493,2017-03-19,249,226,229,179,106,143,0.425703,0.574297,1370,...,0.394681,0.605319,2041.046512,7.743868,0.046512,0.000213,6.627907,0.025808,4.534884,0.017283
3494,2017-03-20,267,243,253,191,109,158,0.408240,0.591760,999,...,0.393486,0.606514,2056.476190,7.797668,0.047619,0.000219,6.619048,0.025766,4.547619,0.017320
3495,2017-03-21,275,256,266,203,99,176,0.360000,0.640000,1240,...,0.394244,0.605756,2032.878049,7.649525,0.048780,0.000224,6.463415,0.024940,4.634146,0.017630
